In [1]:
from pathlib import Path
import enum
from attr import dataclass


In [2]:
with Path("../07.in").open() as f:
    data = f.read().splitlines()


In [3]:
testdata = """\
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483""".splitlines()


# Part I

In [4]:
class Valuation(enum.IntEnum):
    HighCard = 1  # All cards unique
    OnePair = 2  # One pair
    TwoPair = 3  # Two pairs
    ThreeOfAKind = 4  # A Triple
    FullHouse = 5  # A triple and a pair
    FourOfAKind = 6  # Four of a kind
    FiveOfAKind = 7  # Five of a kind


@dataclass(init=False, eq=False)  # eq=True threw the sorting off
class Hand:
    cards: list[int]
    sorted_cards: list[int]
    bet: int
    value: Valuation = None

    __card_values = {
        "A": 14,
        "K": 13,
        "Q": 12,
        "J": 11,
        "T": 10,
        "9": 9,
        "8": 8,
        "7": 7,
        "6": 6,
        "5": 5,
        "4": 4,
        "3": 3,
        "2": 2,
    }


    def __init__(self, cards, bet):
        self.cards = tuple(self.__card_values[c] for c in cards)
        self.sorted_cards = sorted(self.cards, reverse=True)
        self.bet = bet
        self.value = self._get_valuation()

    def _get_valuation(self) -> Valuation:
        card_counts = [self.cards.count(card) for card in self.__card_values.values()]
        max_occurrence = max(card_counts)

        # Unique Option
        if max_occurrence == 1:
            return Valuation.HighCard

        # Could be OnePair or TwoPair
        elif max_occurrence == 2:
            number_of_pairs = card_counts.count(2)
            if number_of_pairs == 1:
                return Valuation.OnePair
            else:
                return Valuation.TwoPair

        # Unique Option
        elif set(card_counts) == {0, 2, 3}:
            return Valuation.FullHouse

        # Unique Option
        elif max_occurrence == 3:
            return Valuation.ThreeOfAKind

        # Unique Option
        elif max_occurrence == 4:
            return Valuation.FourOfAKind

        # Unique Option
        elif max_occurrence == 5:
            return Valuation.FiveOfAKind

    def __lt__(self, other):
        if self.value == other.value:
            return self.cards < other.cards
        else:
            return self.value < other.value

    def __gt__(self, other):
        if self.value == other.value:
            return self.cards > other.cards
        else:
            return self.value > other.value

    def __le__(self, other):
        return self.__eq__(other) or self.__lt__(other)

    def __ge__(self, other):
        return self.__eq__(other) or self.__gt__(other)

    def __eq__(self, other):
        eq = self.value == other.value
        if eq:
            print("Found a duplicate!")
        return eq

    def __str__(self) -> str:
        return f"{self.cards}\t with a bet of \t{self.bet}\t is valued as \t{self.value}"


assert Hand(["2", "3", "4", "5", "6"], 1).value == Valuation.HighCard
assert Hand(["2", "2", "4", "5", "6"], 1).value == Valuation.OnePair
assert Hand(["2", "2", "4", "4", "6"], 1).value == Valuation.TwoPair
assert Hand(["2", "2", "2", "5", "5"], 1).value == Valuation.FullHouse
assert Hand(["2", "2", "2", "5", "6"], 1).value == Valuation.ThreeOfAKind
assert Hand(["2", "2", "2", "2", "6"], 1).value == Valuation.FourOfAKind
assert Hand(["2", "2", "2", "2", "2"], 1).value == Valuation.FiveOfAKind


In [5]:

def parse(lines: list[str], hand_cls) -> list:
    hands: list[hand_cls] = []

    for line in lines:
        card, bet = line.split(" ")
        hand = hand_cls(card, int(bet))
        hands.append(hand)

    return hands


In [6]:
def solve(data, hand_cls, verbose = False):
    total_winnings = 0

    for i, hand in enumerate(sorted(parse(data, hand_cls)), start=1):
        winning = hand.bet * i
        if verbose:
            print(f"{i:02}:\t {hand}\t wins {winning}")
        total_winnings += winning

    return total_winnings


In [7]:
part_1 = solve(testdata, Hand)
print(f"Part 1 Test: {part_1}")
assert part_1 == 6440


Part 1 Test: 6440


In [8]:
part_1 = solve(data, Hand)
print(f"Part 1: {part_1}")


Part 1: 250957639


# Part II - Jokers

The `J` are now Jokers, they can represent any other card:

- `JJQQ2` will be valued as `QQQQ2` -> `FourOfAKind`

But when breaking ties, the `J` counts as `1`:

- `J2222` will lose to `22222` 


In [9]:

@dataclass(init=False, eq=False)  # eq=True threw the sorting off
class JokerHand:
    cards: list[int]
    sorted_cards: list[int]
    bet: int
    value: Valuation = None

    __card_values = {
        "A": 14,
        "K": 13,
        "Q": 12,
        # "J": 11,
        "T": 10,
        "9": 9,
        "8": 8,
        "7": 7,
        "6": 6,
        "5": 5,
        "4": 4,
        "3": 3,
        "2": 2,
        "J": 1
    }


    def __init__(self, cards, bet):
        self.cards = tuple(self.__card_values[c] for c in cards)
        self.sorted_cards = sorted(self.cards, reverse=True)
        self.bet = bet
        self.value = self._get_valuation()

    def _get_valuation(self) -> Valuation:
        card_counts = [self.cards.count(card) for card in self.__card_values.values()]
        joker_count = card_counts.pop()

        max_occurrence = max(card_counts)

        if max_occurrence == 5:
            return Valuation.FiveOfAKind

        elif max_occurrence == 4:
            if joker_count == 1:
                return Valuation.FiveOfAKind
            else:
                return Valuation.FourOfAKind

        elif set(card_counts) == {0, 2, 3}:
            return Valuation.FullHouse

        elif max_occurrence == 3:
            if joker_count == 0:
                return Valuation.ThreeOfAKind
            elif joker_count == 1:
                return Valuation.FourOfAKind
            elif joker_count == 2:
                return Valuation.FiveOfAKind

        elif max_occurrence == 2:
            number_of_pairs = card_counts.count(2)

            # One Pair
            if number_of_pairs == 1:
                if joker_count == 0:
                    return Valuation.OnePair
                elif joker_count == 1:
                    # We could also do TwoPair, but ThreeOfAKind is better
                    return Valuation.ThreeOfAKind
                elif joker_count == 2:
                    return Valuation.FourOfAKind
                elif joker_count == 3:
                    return Valuation.FiveOfAKind

            # Two Pairs
            else:
                if joker_count == 1:
                    # Two Pairs and a joker is a Full House
                    return Valuation.FullHouse
                else:
                    return Valuation.TwoPair

        elif max_occurrence == 1 and joker_count == 0:
            return Valuation.HighCard

        elif joker_count == 1:
            return Valuation.OnePair
        elif joker_count == 2:
            return Valuation.ThreeOfAKind
        elif joker_count == 3:
            return Valuation.FourOfAKind
        elif joker_count >= 4:
            return Valuation.FiveOfAKind


    def __lt__(self, other):
        if self.value == other.value:
            return self.cards < other.cards
        else:
            return self.value < other.value

    def __gt__(self, other):
        if self.value == other.value:
            return self.cards > other.cards
        else:
            return self.value > other.value

    def __le__(self, other):
        return self.__eq__(other) or self.__lt__(other)

    def __ge__(self, other):
        return self.__eq__(other) or self.__gt__(other)

    def __eq__(self, other):
        eq = self.value == other.value
        if eq:
            print("Found a duplicate!")
        return eq

    def __str__(self) -> str:
        return f"{self.cards}\t with a bet of \t{self.bet}\t is valued as \t{self.value}"


# Non-Joker Cases
assert (a := JokerHand(["2", "3", "4", "5", "6"], 1)).value == Valuation.HighCard, a
assert (a := JokerHand(["2", "2", "4", "5", "6"], 1)).value == Valuation.OnePair, a
assert (a := JokerHand(["2", "2", "4", "4", "6"], 1)).value == Valuation.TwoPair, a
assert (a := JokerHand(["2", "2", "2", "5", "5"], 1)).value == Valuation.FullHouse, a
assert (a := JokerHand(["2", "2", "2", "5", "6"], 1)).value == Valuation.ThreeOfAKind, a
assert (a := JokerHand(["2", "2", "2", "2", "6"], 1)).value == Valuation.FourOfAKind, a
assert (a := JokerHand(["2", "2", "2", "2", "2"], 1)).value == Valuation.FiveOfAKind, a

# Joker Cases (J = 1)
assert (a := JokerHand(["J", "2", "2", "2", "2"], 1)).value == Valuation.FiveOfAKind, a
assert (a := JokerHand(["J", "J", "2", "2", "2"], 1)).value == Valuation.FiveOfAKind, a
assert (a := JokerHand(["J", "J", "J", "2", "2"], 1)).value == Valuation.FiveOfAKind, a
assert (a := JokerHand(["J", "J", "J", "J", "2"], 1)).value == Valuation.FiveOfAKind, a
assert (a := JokerHand(["J", "J", "J", "J", "J"], 1)).value == Valuation.FiveOfAKind, a

assert (a := JokerHand(["J", "2", "2", "2", "3"], 1)).value == Valuation.FourOfAKind, a
assert (a := JokerHand(["J", "J", "2", "2", "3"], 1)).value == Valuation.FourOfAKind, a
assert (a := JokerHand(["J", "J", "J", "2", "3"], 1)).value == Valuation.FourOfAKind, a

assert (a := JokerHand(["J", "2", "2", "3", "3"], 1)).value == Valuation.FullHouse, a

assert (a := JokerHand(["J", "2", "2", "3", "4"], 1)).value == Valuation.ThreeOfAKind, a
assert (a := JokerHand(["J", "J", "2", "3", "4"], 1)).value == Valuation.ThreeOfAKind, a

assert (a := JokerHand(["J", "2", "3", "4", "5"], 1)).value == Valuation.OnePair, a


In [10]:
part_2 = solve(testdata, JokerHand)
print(f"Part 2 Test: {part_2}")
assert part_2 == 5905


Part 2 Test: 5905


In [11]:
part_2 = solve(data, JokerHand)
print(f"Part 2: {part_2}")


Part 2: 251515496
